In [1]:
import tensorflow as tf
from prosit_t.wandb_agent.train_utils import get_tfdatasets_padded_filtered
import os
import argparse
import yaml
from dlomix.losses import masked_spectral_distance
import wandb
from prosit_t.models import PrositTransformerV2
from prosit_t.eval import prosit_transformer_eval
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from prosit_t.constants import ALPHABET_UNMOD

2023-11-21 11:14:20.785332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 11:14:20.892301: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 11:14:21.588602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-21 11:14:21.588661: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
project_name = "transforming-prosit-full-dataset"
run = wandb.init(project=project_name)

wandb: Currently logged in as: mamisashvili-lizi (prosit-compms). Use `wandb login --relogin` to force relogin


In [5]:
artifact_path = "prosit-compms/transforming-prosit-full-dataset/model-leafy-universe-28:v26"
model = run.use_artifact(artifact_path)
model_dir = model.download()

wandb:   4 of 4 files downloaded.  


In [6]:
model_loaded = tf.keras.models.load_model(model_dir, custom_objects={
    "masked_spectral_distance": masked_spectral_distance,
})

2023-11-21 11:14:41.279677: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 11:14:41.411806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43607 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [7]:
data_config = {
    "data_source": {
        "train": "/cmnfs/proj/prosit/Transformer/all_unmod_train.parquet",
          "val": "/cmnfs/proj/prosit/Transformer/all_unmod_test.parquet"
    },
    "seq_length": 30,
    "batch_size": 1024
          
}

In [8]:
train_dataset, val_dataset = get_tfdatasets_padded_filtered(data_config)

In [10]:
cb_wandb = WandbCallback()

callback_earlystopping = EarlyStopping(
    monitor="val_loss",
    patience=8,
    min_delta=0.0001,
    restore_best_weights=True,
    verbose=1,
)

callback_reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.8,
    patience=4,
)
callbacks = [cb_wandb, callback_earlystopping, callback_reduce_lr]

In [ ]:
model_loaded.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100


2023-11-21 11:36:04.871537: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45725646848 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-11-21 11:36:06.211663: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


12178/12178 [==============================] - ETA: 0s - loss: 0.1124

2023-11-21 12:06:54.147637: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45725646848 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231121_111426-t5ltsie7/files/model-best/assets


...

12178/12178 [==============================] - 2040s 167ms/step - loss: 0.1124 - val_loss: 0.1449 - lr: 1.0000e-04
Epoch 2/100
12178/12178 [==============================] - ETA: 0s - loss: 0.1121

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231121_111426-t5ltsie7/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231121_111426-t5ltsie7/files/model-best/assets
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231121_111426-t5ltsie7/files/model-best)... Done. 0.2s


12178/12178 [==============================] - 2026s 166ms/step - loss: 0.1121 - val_loss: 0.1434 - lr: 1.0000e-04
Epoch 3/100
12178/12178 [==============================] - 2021s 166ms/step - loss: 0.1116 - val_loss: 0.1448 - lr: 1.0000e-04
Epoch 4/100
12178/12178 [==============================] - 2021s 166ms/step - loss: 0.1113 - val_loss: 0.1491 - lr: 1.0000e-04
Epoch 5/100
 4124/12178 [=========>....................] - ETA: 20:16 - loss: 0.1229

In [14]:
transformer.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 string_lookup (StringLookup)   (None, 30)           0           ['input_4[0][0]']                
                                                                                                  
 positional_embedding (Position  (None, 30, 64)      1472        ['string_lookup[1][0]']          
 alEmbedding)                                                                                     
                                                                                                  
 transformer_encoder (Transform  (None, 30, 64)      2157824     ['positional_embedding[1][0